### HW2 -  Linear Classifier
### Srushti Nayak

#### Importing required libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt

c:\users\asus\appdata\local\programs\python\python39\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Loading CIFAR-10 data from pytorch and applying transformation on data

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


#### Converting to numppy arrays

In [3]:
X_train = np.array([item[0].numpy().flatten() for item in trainset])

In [4]:
X_train.shape

(50000, 3072)

In [5]:
y_train = np.array([item[1] for item in trainset])

In [6]:
y_train.shape

(50000,)

In [7]:
X_test = np.array([item[0].numpy().flatten() for item in testset])

In [8]:
y_test = np.array([item[1] for item in testset])

#### Implementing linear classifier without regularization

In [9]:
class Linear_classifier:
    
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def setHyperParameters(self, epochs, learning_rate, total_classes, input_size):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.weights = np.random.randn(input_size, total_classes) * 0.0001
        self.bias = np.zeros((1, total_classes))
        
    def forward_pass(self, X, w, bias):
        return np.dot(X, w) + bias
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
         
    def cross_entropy_loss(self, y_pred, y_train):
        m = y_train.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_train])
        loss = np.sum(log_likelihood) / m
        return loss
    
    def backward_pass(self, X, y_train, y_pred):
        m = y_train.shape[0]
        grad_softmax = y_pred
        grad_softmax[range(m), y_train] -= 1
        grad_softmax /= m
        grad_weights = np.dot(X.T, grad_softmax)
        grad_bias = np.sum(grad_softmax, axis=0, keepdims=True)
        return grad_weights, grad_bias

    def update_parameters(self, grad_weights, grad_bias):
        self.weights = self.weights - learning_rate * grad_weights
        self.bias = self.bias - learning_rate * grad_bias
        
    def train(self):
        
        for epoch in range(epochs):
            
            output = self.forward_pass(self.X_train, self.weights,self.bias)
            y_pred = self.softmax(output)
            cross_entropy_loss = self.cross_entropy_loss(y_pred, self.y_train)
            grad_weights, grad_bias = self.backward_pass(self.X_train, self.y_train, y_pred)
            self.update_parameters(grad_weights, grad_bias)
            
            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch + 1}, Loss: {cross_entropy_loss}')
    
    def test(self, X_test, y_test):
        
        output = self.forward_pass(X_test, self.weights, self.bias)
        y_pred = self.softmax(output)
        predictions = np.argmax(y_pred, axis=1)
        accuracy = np.mean(predictions == y_test) * 100 
        print(accuracy)

#### defining hyperparameters and applying linear regression to the model

In [10]:
epochs = 100
input_size = 32*32*3
total_classes = 10
learning_rate = 0.01

In [11]:
liner_classifier_without_regularization = Linear_classifier(X_train,y_train)

In [12]:
liner_classifier_without_regularization.setHyperParameters(epochs, learning_rate, total_classes, input_size)

In [13]:
liner_classifier_without_regularization.train()

Epoch 10, Loss: 2.0969283844936224
Epoch 20, Loss: 2.024564744811901
Epoch 30, Loss: 1.982178246335534
Epoch 40, Loss: 1.9529024317851003
Epoch 50, Loss: 1.9310551652515895
Epoch 60, Loss: 1.9139478214258445
Epoch 70, Loss: 1.9000892960581703
Epoch 80, Loss: 1.8885683058601854
Epoch 90, Loss: 1.8787905710346968
Epoch 100, Loss: 1.8703501588161369


In [14]:
liner_classifier_without_regularization.test(X_test, y_test)

36.59


#### Linear classifier with L1 regularization

In [15]:
class Linear_classifier_L1_reg:
    
    # initializing trainset
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def setHyperParameters(self, epochs, learning_rate, total_classes, l1_strength, input_size):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.l1_strength = l1_strength
        self.weights = np.random.randn(input_size, total_classes) * 0.0001
        self.bias = np.zeros((1, total_classes))
        
    def forward_pass(self, X, w):
        return np.dot(X, w) + self.bias
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
         
    def cross_entropy_loss(self, y_pred, y_train):
        m = y_train.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_train])
        loss = np.sum(log_likelihood) / m
        return loss
    
    def backward_pass(self, X, y_train, y_pred):
        m = y_train.shape[0]
        grad_softmax = y_pred
        grad_softmax[range(m), y_train] -= 1
        grad_softmax /= m
        grad_weights = np.dot(X.T, grad_softmax)
        grad_bias = np.sum(grad_softmax, axis=0, keepdims=True)
        return grad_weights, grad_bias

    def update_parameters(self, grad_weights, grad_bias):
        self.weights = self.weights - (self.learning_rate * grad_weights)
        self.bias = self.bias - (self.learning_rate * grad_bias)
        
    def train(self):
        
        for epoch in range(epochs):
            
            output = self.forward_pass(self.X_train, self.weights)
            y_pred = self.softmax(output)
            l1_regularization = self.l1_strength * np.sum(np.abs(self.weights))
            total_loss = self.cross_entropy_loss(y_pred, self.y_train) + l1_regularization
            grad_weights, grad_bias = self.backward_pass(self.X_train, self.y_train, y_pred)
            grad_weights = grad_weights + ((self.l1_strength/self.X_train.shape[0]) * np.sign(self.weights))
            self.update_parameters(grad_weights, grad_bias)
            
            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch + 1}, Loss: {total_loss}')
    
    def test(self, X_test, y_test):
        
        output = self.forward_pass(X_test, self.weights)
        y_pred = self.softmax(output)
        predictions = np.argmax(y_pred, axis=1)
        accuracy = np.mean(predictions == y_test) * 100 
        print(accuracy)

#### defining hyperparameters and applying linear regression to the model

In [16]:
epochs = 100
input_size = 32*32*3
total_classes = 10
learning_rate = 0.01
l1_strength = 0.001

In [17]:
L1_regularizated_model = Linear_classifier_L1_reg(X_train, y_train)

In [18]:
L1_regularizated_model.setHyperParameters(epochs, learning_rate, total_classes, l1_strength, input_size)

In [19]:
L1_regularizated_model.train()

Epoch 10, Loss: 2.1148008933834697
Epoch 20, Loss: 2.0532408211226745
Epoch 30, Loss: 2.0192267699225233
Epoch 40, Loss: 1.9968820119349238
Epoch 50, Loss: 1.9809770471509422
Epoch 60, Loss: 1.9690880822570616
Epoch 70, Loss: 1.95988775667998
Epoch 80, Loss: 1.9525712915296314
Epoch 90, Loss: 1.9466317200885108
Epoch 100, Loss: 1.941723356799024


In [20]:
L1_regularizated_model.test(X_test, y_test)

36.63


#### Linear classifier with L2 regularization

In [21]:
class Linear_classifier_L2_reg:
    
    # initializing trainset
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def setHyperParameters(self, epochs, learning_rate, total_classes, l2_strength, input_size):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.l2_strength = l2_strength
        self.weights = np.random.randn(input_size, total_classes) * 0.0001
        self.bias = np.zeros((1, total_classes))
        
    def forward_pass(self, X, w):
        return np.dot(X, w) + self.bias
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
         
    def cross_entropy_loss(self, y_pred, y_train):
        m = y_train.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_train])
        loss = np.sum(log_likelihood) / m
        return loss 
    
    def backward_pass(self, X, y_train, y_pred):
        m = y_train.shape[0]
        grad_softmax = y_pred
        grad_softmax[range(m), y_train] -= 1
        grad_softmax /= m
        grad_weights = np.dot(X.T, grad_softmax)
        grad_bias = np.sum(grad_softmax, axis=0, keepdims=True)
        return grad_weights, grad_bias

    def update_parameters(self, grad_weights, grad_bias):
        self.weights = self.weights - learning_rate * grad_weights
        self.bias = self.bias - learning_rate * grad_bias
        
    def train(self):
        
        for epoch in range(epochs):
            
            output = self.forward_pass(self.X_train, self.weights)
            y_pred = self.softmax(output)
            l2_regularization = 0.5 * self.l2_strength * np.sum(self.weights ** 2)
            total_loss = self.cross_entropy_loss(y_pred, self.y_train) + l2_regularization
            grad_weights, grad_bias = self.backward_pass(self.X_train, self.y_train, y_pred)
            grad_weights = grad_weights + ((self.l2_strength/self.X_train.shape[0]) * self.weights)
            self.update_parameters(grad_weights, grad_bias)
            
            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch + 1}, Loss: {total_loss}')
    
    def test(self, X_test, y_test):
        
        output = self.forward_pass(X_test, self.weights)
        y_pred = self.softmax(output)
        predictions = np.argmax(y_pred, axis=1)
        accuracy = np.mean(predictions == y_test) * 100 
        print(accuracy)

#### defining hyperparameters and applying linear regression to the model

In [22]:
epochs = 100
input_size = 32*32*3
total_classes = 10
learning_rate = 0.01
l2_strength = 0.001

In [23]:
L2_regularizated_model = Linear_classifier_L2_reg(X_train, y_train)

In [24]:
L2_regularizated_model.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, input_size)

In [25]:
L2_regularizated_model.train()

Epoch 10, Loss: 2.096936524143008
Epoch 20, Loss: 2.024587661917383
Epoch 30, Loss: 1.9822163851384886
Epoch 40, Loss: 1.9529554578706227
Epoch 50, Loss: 1.9311229142298898
Epoch 60, Loss: 1.9140300549419642
Epoch 70, Loss: 1.9001857024903595
Epoch 80, Loss: 1.8886785288519177
Epoch 90, Loss: 1.8789142335273004
Epoch 100, Loss: 1.870486880134562


In [26]:
L2_regularizated_model.test(X_test, y_test)

36.65


#### Linear classifier with Elastic net regularization

In [27]:
class Linear_classifier_Elastic_net_reg:
    
    # initializing trainset
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def setHyperParameters(self, epochs, learning_rate, total_classes, l1_strength, l2_strength, input_size):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.l2_strength = l2_strength
        self.l1_strength = l1_strength
        self.weights = np.random.randn(input_size, total_classes) * 0.0001
        self.bias = np.zeros((1, total_classes))
        
    def forward_pass(self, X, w):
        return np.dot(X, w) + self.bias
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
         
    def cross_entropy_loss(self, y_pred, y_train):
        m = y_train.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_train])
        loss = np.sum(log_likelihood) / m
        return loss
    
    def backward_pass(self, X, y_train, y_pred):
        m = y_train.shape[0]
        grad_softmax = y_pred
        grad_softmax[range(m), y_train] -= 1
        grad_softmax /= m
        grad_weights = np.dot(X.T, grad_softmax)
        grad_bias = np.sum(grad_softmax, axis=0, keepdims=True)
        return grad_weights, grad_bias

    def update_parameters(self, grad_weights, grad_bias):
        self.weights = self.weights - learning_rate * grad_weights
        self.bias = self.bias - learning_rate * grad_bias
        
    def train(self):
        
        for epoch in range(epochs):
            
            output = self.forward_pass(self.X_train, self.weights)
            y_pred = self.softmax(output)
            l1_regularization = self.l1_strength * np.sum(np.abs(self.weights))
            l2_regularization = 0.5 * self.l2_strength * np.sum(self.weights ** 2)
            total_loss = self.cross_entropy_loss(y_pred, self.y_train) + l2_regularization + l1_regularization
            grad_weights, grad_bias = self.backward_pass(self.X_train, self.y_train, y_pred)
            grad_weights = grad_weights + (self.l2_strength * self.weights) + ((self.l1_strength/self.X_train.shape[0]) * np.sign(self.weights))
            self.update_parameters(grad_weights, grad_bias)
            
            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch + 1}, Loss: {total_loss}')
    
    def test(self, X_test, y_test):
        
        output = self.forward_pass(X_test, self.weights)
        y_pred = self.softmax(output)
        predictions = np.argmax(y_pred, axis=1)
        accuracy = np.mean(predictions == y_test) * 100 
        print(accuracy)

#### defining hyperparameters and applying linear regression to the model

In [28]:
epochs = 100
input_size = 32*32*3
total_classes = 10
learning_rate = 0.01
l2_strength = 0.001
l1_strength = 0.001

In [29]:
Elastic_net_regularizated_model = Linear_classifier_Elastic_net_reg(X_train, y_train)

In [30]:
Elastic_net_regularizated_model.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, l1_strength, input_size)

In [31]:
Elastic_net_regularizated_model.train()

Epoch 10, Loss: 2.1148000638974196
Epoch 20, Loss: 2.053295150721024
Epoch 30, Loss: 2.019308280281109
Epoch 40, Loss: 1.9969778161678378
Epoch 50, Loss: 1.981089256932737
Epoch 60, Loss: 1.9692172078041092
Epoch 70, Loss: 1.9600370963981186
Epoch 80, Loss: 1.9527441137377186
Epoch 90, Loss: 1.9468193032607963
Epoch 100, Loss: 1.9419235910178454


In [32]:
Elastic_net_regularizated_model.test(X_test, y_test)

36.68


#### 1)Hyperparameter tuning, learning_rate = 0.05 and l2_strength =0.001 epochs = 100

In [33]:
epochs = 100
input_size = 32*32*3
total_classes = 10
learning_rate = 0.05
l2_strength = 0.001

In [34]:
L2_regularizated_model_2 = Linear_classifier_L2_reg(X_train, y_train)

In [35]:
L2_regularizated_model_2.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, input_size)

In [36]:
L2_regularizated_model_2.train()

Epoch 10, Loss: 1.9361919787675559
Epoch 20, Loss: 1.8723947896384951
Epoch 30, Loss: 1.841754304692046
Epoch 40, Loss: 1.8223956863764237
Epoch 50, Loss: 1.8084357610026562
Epoch 60, Loss: 1.7975690183705746
Epoch 70, Loss: 1.788681322988357
Epoch 80, Loss: 1.7811591264769988
Epoch 90, Loss: 1.7746328780047926
Epoch 100, Loss: 1.7688647263313106


In [37]:
L2_regularizated_model_2.test(X_test, y_test)

39.6


#### 2_1) Hyperparameter tuning, learning_rate = 0.05 and l2_strength =0.001 epochs = 200

In [38]:
epochs = 200
input_size = 32*32*3
total_classes = 10
learning_rate = 0.05
l2_strength = 0.001

In [39]:
L2_regularizated_model_2_1 = Linear_classifier_L2_reg(X_train, y_train)

In [40]:
L2_regularizated_model_2_1.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, input_size)

In [41]:
L2_regularizated_model_2_1.train()

Epoch 10, Loss: 1.9361753654664873
Epoch 20, Loss: 1.872394581535456
Epoch 30, Loss: 1.8417618540048166
Epoch 40, Loss: 1.8224074415429055
Epoch 50, Loss: 1.8084497416993404
Epoch 60, Loss: 1.7975840462129833
Epoch 70, Loss: 1.788696693459349
Epoch 80, Loss: 1.7811744232040736
Epoch 90, Loss: 1.7746478610494063
Epoch 100, Loss: 1.768879264042551
Epoch 110, Loss: 1.763707480546505
Epoch 120, Loss: 1.7590186095170117
Epoch 130, Loss: 1.754729310639533
Epoch 140, Loss: 1.7507767990843897
Epoch 150, Loss: 1.7471125804434444
Epoch 160, Loss: 1.7436983807172737
Epoch 170, Loss: 1.7405034179002796
Epoch 180, Loss: 1.7375025226245693
Epoch 190, Loss: 1.7346748125827662
Epoch 200, Loss: 1.7320027377257265


In [42]:
L2_regularizated_model_2_1.test(X_test, y_test)

40.6


#### 2) Hyperparameter tuning, learning_rate = 0.05 and l2_strength =0.008 and epoch = 200

In [43]:
epochs = 200
input_size = 32*32*3
total_classes = 10
learning_rate = 0.05
l2_strength = 0.008

In [44]:
L2_regularizated_model_3 = Linear_classifier_L2_reg(X_train, y_train)

In [45]:
L2_regularizated_model_3.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, input_size)

In [46]:
L2_regularizated_model_3.train()

Epoch 10, Loss: 1.936637503615806
Epoch 20, Loss: 1.873325840745382
Epoch 30, Loss: 1.8431068627271214
Epoch 40, Loss: 1.8241232683914383
Epoch 50, Loss: 1.8105059930280765
Epoch 60, Loss: 1.799958913488967
Epoch 70, Loss: 1.7913741142084367
Epoch 80, Loss: 1.7841422551623467
Epoch 90, Loss: 1.7778967017406386
Epoch 100, Loss: 1.7724016709256067
Epoch 110, Loss: 1.7674974435192408
Epoch 120, Loss: 1.7630711858325718
Epoch 130, Loss: 1.7590403666308236
Epoch 140, Loss: 1.7553428236284554
Epoch 150, Loss: 1.7519305476274996
Epoch 160, Loss: 1.748765646955843
Epoch 170, Loss: 1.7458176436723336
Epoch 180, Loss: 1.7430616121539892
Epoch 190, Loss: 1.7404768668387118
Epoch 200, Loss: 1.738046017477062


In [47]:
L2_regularizated_model_3.test(X_test, y_test)

40.61


#### 3) Hyperparameter tuning, learning_rate = 0.08 and l2_strength =0.01 and epoch = 200

In [48]:
epochs = 200
input_size = 32*32*3
total_classes = 10
learning_rate = 0.08
l2_strength = 0.01

In [49]:
L2_regularizated_model_4 = Linear_classifier_L2_reg(X_train, y_train)

In [50]:
L2_regularizated_model_4.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, input_size)

In [51]:
L2_regularizated_model_4.train()

Epoch 10, Loss: 1.901802214386546
Epoch 20, Loss: 1.8550643236504945
Epoch 30, Loss: 1.8290737455960873
Epoch 40, Loss: 1.8103163782946268
Epoch 50, Loss: 1.7963182569481493
Epoch 60, Loss: 1.7852497149049797
Epoch 70, Loss: 1.7761013539701003
Epoch 80, Loss: 1.7683074841829214
Epoch 90, Loss: 1.7615273081604375
Epoch 100, Loss: 1.7555413679066474
Epoch 110, Loss: 1.7502003692576185
Epoch 120, Loss: 1.7453981455111067
Epoch 130, Loss: 1.741056481223965
Epoch 140, Loss: 1.737116102300407
Epoch 150, Loss: 1.7335309962269119
Epoch 160, Loss: 1.7302645581880718
Epoch 170, Loss: 1.7272867342811016
Epoch 180, Loss: 1.724571725333625
Epoch 190, Loss: 1.7220960913212373
Epoch 200, Loss: 1.7198373104088787


In [52]:
L2_regularizated_model_4.test(X_test, y_test)

40.92


#### 4) Hyperparameter tuning, learning_rate = 0.1 and l2_strength =0.05 and epoch = 200

In [53]:
epochs = 200
input_size = 32*32*3
total_classes = 10
learning_rate = 0.1
l2_strength = 0.05

In [54]:
L2_regularizated_model_5 = Linear_classifier_L2_reg(X_train, y_train)

In [55]:
L2_regularizated_model_5.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, input_size)

In [56]:
L2_regularizated_model_4.train()

Epoch 10, Loss: 1.8246386100282188
Epoch 20, Loss: 1.8570471974176934
Epoch 30, Loss: 1.8245493234876502
Epoch 40, Loss: 1.8078264983792118
Epoch 50, Loss: 1.7962322791288885
Epoch 60, Loss: 1.7954034446238525
Epoch 70, Loss: 1.902244401422413
Epoch 80, Loss: 1.8658989217080193
Epoch 90, Loss: 1.7941933048065102
Epoch 100, Loss: 1.772268532131215
Epoch 110, Loss: 1.783651673834355
Epoch 120, Loss: 1.8356816885906109
Epoch 130, Loss: 1.8119042252531996
Epoch 140, Loss: 1.7684065392860866
Epoch 150, Loss: 1.7669647907771806
Epoch 160, Loss: 1.7748381850478219
Epoch 170, Loss: 1.7849427019694726
Epoch 180, Loss: 1.7843237191553127
Epoch 190, Loss: 1.765706149143236
Epoch 200, Loss: 1.7600607788086509


In [57]:
L2_regularizated_model_4.test(X_test, y_test)

39.51


#### 5) Hyperparameter tuning, learning_rate = 0.08 and l2_strength =0.09 and epoch = 200

In [58]:
epochs = 200
input_size = 32*32*3
total_classes = 10
learning_rate = 0.08
l2_strength = 0.09

In [59]:
L2_regularizated_model_6 = Linear_classifier_L2_reg(X_train, y_train)

In [60]:
L2_regularizated_model_6.setHyperParameters(epochs, learning_rate, total_classes, l2_strength, input_size)

In [61]:
L2_regularizated_model_6.train()

Epoch 10, Loss: 1.9100935298664912
Epoch 20, Loss: 1.8711670331859431
Epoch 30, Loss: 1.851710456634327
Epoch 40, Loss: 1.8387760607322887
Epoch 50, Loss: 1.830166057522058
Epoch 60, Loss: 1.8242019168782653
Epoch 70, Loss: 1.819962113804135
Epoch 80, Loss: 1.8169346505178983
Epoch 90, Loss: 1.8148130533040514
Epoch 100, Loss: 1.8134006457226546
Epoch 110, Loss: 1.8125637149925546
Epoch 120, Loss: 1.8122070098064997
Epoch 130, Loss: 1.8122601163742944
Epoch 140, Loss: 1.812669440623826
Epoch 150, Loss: 1.813393184654593
Epoch 160, Loss: 1.8143979376223067
Epoch 170, Loss: 1.815656124497563
Epoch 180, Loss: 1.8171439201613862
Epoch 190, Loss: 1.8188394971812998
Epoch 200, Loss: 1.8207216810723585


In [62]:
L2_regularizated_model_6.test(X_test, y_test)

40.93


#### 6) Hyperparameter tuning, learning_rate = 0.08 and epoch = 200 with linear model without regularization

In [63]:
epochs = 200
input_size = 32*32*3
total_classes = 10
learning_rate = 0.08

In [64]:
liner_classifier_without_regularization_2 = Linear_classifier(X_train,y_train)

In [65]:
liner_classifier_without_regularization_2.setHyperParameters(epochs, learning_rate, total_classes, input_size)

In [66]:
liner_classifier_without_regularization_2.train()

Epoch 10, Loss: 1.9006792823785386
Epoch 20, Loss: 1.8529873925410258
Epoch 30, Loss: 1.8262369641104448
Epoch 40, Loss: 1.8067591584518823
Epoch 50, Loss: 1.7920873318135002
Epoch 60, Loss: 1.7803801088251001
Epoch 70, Loss: 1.7706177206064813
Epoch 80, Loss: 1.7622277578933252
Epoch 90, Loss: 1.7548650865599427
Epoch 100, Loss: 1.7483073643548652
Epoch 110, Loss: 1.7424033223320514
Epoch 120, Loss: 1.7370454089021956
Epoch 130, Loss: 1.7321544177569679
Epoch 140, Loss: 1.7276703528415394
Epoch 150, Loss: 1.7235466659463494
Epoch 160, Loss: 1.719746346972083
Epoch 170, Loss: 1.7162390289512828
Epoch 180, Loss: 1.7129986656608094
Epoch 190, Loss: 1.710001618001369
Epoch 200, Loss: 1.7072252003900252


In [67]:
liner_classifier_without_regularization_2.test(X_test, y_test)

40.94


#### 7) Hyperparameter tuning, l1_strength = 0.001, learning_rate = 0.08 and epoch = 200 with linear model with L1 regularization

In [68]:
epochs = 200
input_size = 32*32*3
total_classes = 10
learning_rate = 0.08
l1_strength = 0.09

In [69]:
L1_regularizated_model_7 = Linear_classifier_L1_reg(X_train, y_train)

In [70]:
L1_regularizated_model_7.setHyperParameters(epochs, learning_rate, total_classes, l1_strength, input_size)

In [71]:
L1_regularizated_model_7.train()

Epoch 10, Loss: 7.535921929769115
Epoch 20, Loss: 9.714207088083983
Epoch 30, Loss: 11.11025620830869
Epoch 40, Loss: 12.162764538332011
Epoch 50, Loss: 13.032807217147104
Epoch 60, Loss: 13.790142721257883
Epoch 70, Loss: 14.469212084720864
Epoch 80, Loss: 15.08987181198009
Epoch 90, Loss: 15.666066347674036
Epoch 100, Loss: 16.206818606348314
Epoch 110, Loss: 16.71837019335496
Epoch 120, Loss: 17.205722078815285
Epoch 130, Loss: 17.672739161643282
Epoch 140, Loss: 18.122158010637058
Epoch 150, Loss: 18.555660278514193
Epoch 160, Loss: 18.975647332672075
Epoch 170, Loss: 19.38346993202391
Epoch 180, Loss: 19.779862791013663
Epoch 190, Loss: 20.164886754020817
Epoch 200, Loss: 20.53945465643307


In [72]:
L1_regularizated_model_7.test(X_test, y_test)

40.89
